In [1]:
import pandas as pd
import os
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import geopandas as gpd
import contextily as ctx
import matplotlib as mpl
# from pandarallel import pandarallel

In [ ]:
# # def get_county_report(county_fip):
# county_address_dir = '../data/county_address'
# for file in tqdm(os.listdir(county_address_dir)):
#     test_county= os.path.join(county_address_dir, file)
#     tdf = pd.read_csv(test_county)
#     null_count = tdf[tdf['address'].isnull()].size
#     if null_count > 0:
#         print(file)

# Generate Empty (Go through each county. If there is a file, join on geoid, if there is not, create empty columns)

In [2]:
prefix = 'https://www2.census.gov/geo/tiger/TIGER2020PL/LAYER/TABBLOCK/2020/'
r= requests.get(prefix)
soup = BeautifulSoup(r.content)
valid_zips = {}

for a in soup.find_all('a', href=True):
    if not '.zip' in a['href'] or len(a['href'])!= 28: # filter for the entire state ones
        continue
    valid_zips[a['href'].split('_')[2]] = prefix+a['href']
print(len(valid_zips))

3221


# Plot Jefferson County (01073)

In [ ]:
jc_df = pd.read_csv('../data/county_address/01073.csv.xz', dtype={'geoid20':object})
county_df = gpd.read_file(valid_zips['01073'])

In [ ]:
jc_df = jc_df.rename(columns={'geoid20': 'GEOID20'})
jc_df

In [ ]:
df = pd.merge(jc_df,county_df[['GEOID20']],on='GEOID20', how='right')
df

In [ ]:
df[df['address'].isnull()]

In [ ]:
# county_df.crs= 4326
# ax = county_df.plot(figsize=(10, 10), alpha=0.7, column='address_count', legend=True)
# ctx.add_basemap(ax, zoom=10, crs=county_df.crs.to_string(), source=ctx.providers.Stamen.Toner)

In [24]:
for county in tqdm(valid_zips):
    if os.path.isfile('../data/address/%s.csv.xz' % county):
        # skip if file already exists
        continue
        
    county_shape_df = gpd.read_file(valid_zips[county])    
    if os.path.isfile('../data/county_address/%s.csv.xz' % county):
        cdf = pd.read_csv('../data/county_address/%s.csv.xz' % county, dtype={'geoid20':object})
        cdf = cdf.rename(columns={'geoid20': 'GEOID20'})
        mdf = pd.merge(cdf,county_shape_df[['GEOID20']],on='GEOID20', how='right') # create empty rows based on authority shape fiels
    else:
        mdf = county_shape_df[['GEOID20']]
        mdf = mdf.reindex(columns= ['address', 'GEOID20', 'longitude', 'latitude'])
        assert mdf.size > 16
    mdf.to_csv('../data/address/%s.csv.xz' % county, index =False)

100%|██████████| 3221/3221 [28:24<00:00,  1.89it/s]  


# Generate Report

In [10]:
# # Do some clean up
# d = '../data/address'
# for file in tqdm(os.listdir(d)):
#     df = pd.read_csv(os.path.join(d,file))
#     if not set(df.columns) == set(['address', 'GEOID20', 'longitude', 'latitude']):    
#         df = df.reindex(columns= ['address', 'GEOID20', 'longitude', 'latitude'])
#         df.to_csv(os.path.join(d,file), index =False)

# #     print(df.columns)
# #     null_count =  df[df['address'].isnull()].size
# #     if null_count > 0:
# #         print(file)

100%|██████████| 3221/3221 [32:54<00:00,  1.63it/s]  


In [25]:
# Do some clean up
d = '../data/address'
for file in tqdm(os.listdir(d)):
    df = pd.read_csv(os.path.join(d,file))
    assert set(df.columns) == set(['address', 'GEOID20', 'longitude', 'latitude'])
    if df.size < 20:
        print('%s: %s' % (file, df.size))
        print(df)
        os.remove(os.path.join(d,file)) 
#     df = df.reindex(columns= ['address', 'GEOID20', 'longitude', 'latitude'])
#     df.to_csv(os.path.join(d,file), index =False)

100%|██████████| 3221/3221 [02:07<00:00, 25.26it/s]


# Generate Report

In [30]:
# Do some clean up
d = '../data/address'
dfs= []
for file in tqdm(os.listdir(d)):
    df = pd.read_csv(os.path.join(d,file))
    report_d = {
        'fip': file,
        'perc_coverage': (df[~df['address'].isnull()].size/df.size)*100, 
        'address_median': df['address'].value_counts().median(),
        'address_std':df['address'].value_counts().std(),     
    }
    report_df = pd.DataFrame(report_d, index=[0])
    dfs.append(report_df)

final_df = pd.concat(dfs)
final_df

100%|██████████| 3221/3221 [02:56<00:00, 18.29it/s]


,fip,perc_coverage,address_median,address_std
0,29085.csv.xz,0.546448,1.0,0.000000
0,09001.csv.xz,99.519541,1.0,0.391420
0,29067.csv.xz,0.279135,1.0,0.000000
0,29073.csv.xz,22.005571,1.0,0.643564
0,21185.csv.xz,0.000000,NaN,NaN
...,...,...,...,...
0,20115.csv.xz,93.413874,1.0,0.104286
0,29131.csv.xz,17.873652,1.0,1.762397
0,31013.csv.xz,0.000000,NaN,NaN
0,31103.csv.xz,0.000000,NaN,NaN


In [ ]:
final_df.sort_values(by=)